### План
* Обучение и чиcтая оптимизация   
* Проблемы градиентного спуска 
* Методы оптимизации


## Обучение vs чистая оптимизация
Алгоритмы оптимизации, используемые для обучения глубоких моделей, отличаются от традиционных алгоритмов оптимизации:

* Нас интересует некоторая мера качества $ P $, определяется относительно тестового набора и может оказаться вычислительно сложной,   мы оптимизируем $ P $ косвенно. Мы уменьшаем другую функцию стоимости $ J(θ) $ в надежде, что при этом улучшится и P.
* Алгоритмы оптимизации для обучения глубоких моделей обычно включают специализации для конкретной структуры целевых функций.


### Минимизация эмпирического риска
Большинство алгоритмов машинного обучения в том или ином виде включает оптимизацию, т. е. нахождение минимума или максимума функции.
Обычно задачу оптимизации формулируют в терминах нахождения минимума функции потерь/стоимости/ошибок – $ J(f(x, \theta), y ) $.  $ \theta^* = \underset{\theta}{\operatorname{arg min}} J( f(x;\theta), y) $ - где $ \theta^* $ - целевые параметры алгоритма

Типичную функцию стоимости можно представить в виде среднего по обучающему набору:

$ J(\theta) = \mathop{\mathbb{E}}_{(x,y)\sim{\hat p_{data}}} L( f(x, \theta), у) $

* $ L $  – функция потерь на одном примере
* $ f(x; θ) $  – предсказанный выход для входа $ x $, 
* $ \hat p_{data} $ – эмпирическое распределение.

Обычно предпочитают минимизировать функцию где математическое ожидание берется по порождающему распределению pdata, а не просто по конечному обучающему набору:

$ J(\theta)^* = \mathop{\mathbb{E}}_{(x,y)\sim{p_{data}}} L( f(x, \theta), у) $

* $ \mathop{\mathbb{E}}_{(x,y)\sim{p_{data}}} $  - матожидание ошибки обобщения по истинному распределению $ p_{data} $ - *_риск_* , мы хотим его уменьшить.

* при известном $ p_{data} $ - минимизация риска $ \mathop{\mathbb{E}}_{(x,y)\sim{p_{data}}} $ - задача оптимизации  
* мы заменяем $ p(x, y) $  на $ \hat p(х, у) $ - посчитанному по обучающему сету. 
 $$ => $$

* минимизируем *_эмпирический риск_*, т.е. ошибку обобщения на обучающем сете $ \large \mathop{\mathbb{E}}_{(x,y)\sim{\hat p_{data}(x, y)}} [L(f(x;\theta), y)]= {1\over m}  \sum \limits_{i=1} ^m {L(f(x^i; \theta), y)}$

m - количество обучающих примеров

Таким образом, мы оптимизируем не риск, а эмпирический риск.



### Сурогатные функции потерь
Иногда реально интересующая нас функция потерь и та, что может быть эффективно оптимизирована, – «две большие разницы».

Пример:
* $ \large R(f) = \mathop{\mathbb{E}}[\mathop{\mathbb{1}}(sign(f(X)) \ne Y )]. $ -  функция риска для бинарной классификации 
* $ \large CE = - \sum \limits_{x} {y * log(p(x, \theta))} $ - кроссэнтропия по классам

В некоторых случаях суррогатная функция потерь позволяет достичь даже больших успехов в обучении. Ошибка может падать на тесте даже если на обучающем сете мы получили 0


### Пакеты и мини-пакеты
В обучении целевая функция представления в виде суммы по обучающим примерам, т.е. ошибка вычисляется не по всему сету (пакет) а по какомуто подмножеству обучающих примеров. Затем вычисляем градиент и обновление параметров  
Пример,оценка максимального правдоподобия:
* $ \large \theta_{ML} = \underset{\theta}{\operatorname{arg max}} \sum \limits_{i=1}^m p_{model}(x^i, y^i; \theta) $

Максимизация эквивлентна максимизации матожидания эмпирического распределения функции стоимости: 
* $ \large J(\theta) = \mathop{\mathbb{E}}_{(x,y)\sim \hat p_{data}} log \space p_{model}(x, y; \theta)$

Большинство свойств целевой функции, используемой в алгоритмах оптимизации, выражаются в терминах мат ожидания:
* $ \large \nabla_{\theta} J(\theta) =  \mathop{\mathbb{E}}_{(x,y)\sim \hat p_{data}} \nabla_{\theta}log \space p_{model}(x, y; \theta) $

#### Важно, что на практике вычисление точного градиента по полному сету будет стоить очень дорого  

Стандартная ошибка среднего оцененная по выборке $ n $:
* $ \large SE(\hat \mu) = \sqrt{Var \Bigg [{{1\over{m}} \sum \limits_{i=1}^m{x^i}}} \Bigg] = {\sigma \over \sqrt{m}} $

!!! точность оценки градиента с увеличением объема выборки растет медленнее, чем линейно. Увеличивая размер выборки в 1000, замедлимся в 1000 раз, но получаем оценку нрадиента всего лишь в 10 раз точнее,

Вторая причина работать мини батчами - наличие повторяющихся примеров в обучающем сете, которые дают похожий вклад в градиент

Для обучения на практике нужно выбиратьчто что-то среднее между пакетным и онлайн обучением.



#### На размер мини-пакета оказывают влияние следующие факторы:
    
* чем больше пакет, тем точнее оценка градиента, но зависимость хуже линейной;
* если пакет очень мал, то не удается в полной мере задействовать преимущества многоядерной архитектуры.
* Для многих аппаратных конфигураций размер пакета – ограничивающий фактор;
* небольшие пакеты могут дать эффект регуляризации (Wilson and Martinez, 2003), быть может, из-за шума, который они вносят в процесс обучения.
* Ошибка обобщения часто оказывается наилучшей для пакета размера 1. Но для обучения с таким маленьким размером пакета нужна небольшая скорость обучения для обеспечения устойчивости из-за высокой дисперсии оценки градиента. 


#### Обучение минипакетами
* В зависимости от алгоритма обучения будет использоваться различная информация, например, методы первого порядка более устойчивы к шуму и могут работать на пакетах в 100 примерах, методы второго порядка менее устойчивы и требуют 10000 примеров в пакете.
* нужно максимально убирать зависимости между примерами и пакетами, для вычисления несмещенной оценки градиента. Для этого нужно выбирать мини-пакеты слцчайно.
* для большого количества данных с миллиардами примеров, сложно получитьвыборку пакетов понастоящему случайно, поэтому набор перемешивают один раз и втаком виде подают алгоритму. 


### Проблемы оптимизации 

#### Плохая обусловленность
* Самая известная проблема это плохая обусловленность матрицы Гессе: $ \large H $, приводит к застреванию SGD,когда малые шаги увеличивают функцию стоимости.

Матрица Гессе для функции нескольких переменных: $ \large H(f(x)_{i,j}) = \frac {\partial^2} {\partial x_i\partial x_j}f(x) $

Если функцию, которую мы апроксимируем разложить в ряд тейлора до членов второго порядка в окрестности $ x_0$ :

$\large f(x) \approx f(x_0) + (x - x_0)^T \space g + \frac{1}{2} \space (x-x_0)^T \space H \space (x-x_0) $

* $  g $ - градиент, а $  H$ - гессиан в точке  $ x_0 $
* если $ \epsilon $ - шаг обучения, то новыя точка $ x = x_0 - \epsilon \space g$

$\large f(x - \epsilon g) \approx f(x_0) -  \epsilon g^T g + \frac{1}{2} \epsilon^2 g^TH g $

Можно видеть, что шаг градиентного спуска $ -\epsilon g $ увеличивает стоимость на $ \frac{1}{2} \epsilon^2 g^TH g - \epsilon g^T g $ 

Плохая обусловленность градиента становится проблемой, когда  $ \frac{1}{2} \epsilon^2 g^TH g $ больше $ \epsilon g^T g $ . 

* квадрат нормы градиента $ g^T g $ и член $  g^THg $ - могут служить мониторингом плохой обусловленности градиента
* норма градиента не сильно уменьшается за время обучения, тогда как член $ g^⏉Hg $ растет на порядок
* В результате обучение происходит очень медленно, несмотря на большой градиент, т. к. приходится уменьшать скоростьобучения, чтобы компенсировать еще большую кривизну функции.

    <img src="./img/gradient_grow.png" width=800>
    
Градиентный спуск часто не находит никакой критической
точки. В этом примере норма градиента возрастает на протяжении всего
процесса обучения  нейронной сети. По идее градиент должен уменьшаться если алгоритм сходится к критической точке.


#### Локалтные минимумы
* Выпуклую оптимизацию можно свести к нахождению локального минимума. Гарантируется, что любой локальный минимум является глобальным
* У нейронных сетей (невыпуклых функций) может быть множество локальных минимумов из-за проблемы __идентифицируемости модели__.

_Модель иентифицируема, если существует достаточно большой набор обучающих данных, который может исключить  все конфигурации параметров кроме одной._ 
* модели с латеньными переменными часто не являются идентифицируемыми. Можно  получить эквивалентные модели, меняя латентные переменные местами. 
* проблема идентифицируемости модели означает, что функция стоимости нейронной сети может иметь очень большое, даже несчетное, множество локальных минимумов
* если значение функции стоимости велико в локально минимуме, то это проблема, но в большинстве случаев обучение находит локальный минимум с низкой стоимостью, и это не является проблеммой

<img src="./img/local_minima.png" width=800>

#### Седловые точки и плато
<img src="./img/saddle_point.png" width=500>

* Для многих невыпуклых функций в многомерных пространствах локальные минимумы (и максимумы) встречаются гораздо реже других точек с нулевым градиентом: седловых точек. В одних точках в окрестности седловой стоимость выше, чем в седловой точке, в других – ниже.

* В седловой точке матрица Гессе имеет как положительные, так и отрицательные собственные значения.

* В точках, лежащих вдоль собственных векторов с положительными собственными значениями, стоимость выше, чем в седловой точке, а в точках, лежащих вдоль собственных векторов с отрицательными собственными значениями, – ниже.

<img src="./img/saddle_point_2D.png" width=500>

* Для функции $ f : ℝ^n → ℝ $ в пространстве высокой размерности ожидаемое отношение числа седловых точек к числу локальных максимумов растет экспоненциально с ростом n. 

* Градиент часто оказывается очень мал в окрестности седловой точки. С другой стороны, есть эмпирические свидетельства в пользу того, что метод градиентного спуска во многих случаях способен выйти из седловой точки.

#### Утесы и резко расстущие градиенты
<img src="./img/exploding_grad.png" width=500>

* В нейронных сетях с большим числом слоев часто встречаются очень крутые участки, напоминающие утесы. Это связано с перемножением нескольких больших весов, например, рекурентные сети. 
* На стене особенно крутого утеса шаг обновления градиента может привести к очень сильному изменению параметров, что обычно заканчивается «срывом».

__Например__
* предположим, что граф вычислений содержит путь, состоящий из повторных умножений на матрицу $ W $
* через t - шагов получаем матрицу $ W^t $ 
* спектральное разложение W имеет вид $ W = V diag(λ)V^{-1} $.
* $ W^t = (V diag(λ)V^{-1})^t = V diag(λ)^t V^–1 $
* все $ \lambda_i $ либо возрастают при $ \lambda_i > 1 $ - __взрыв__ либо __затухают__ при $ \lambda_i < 1 $ 

## Основные алгоритмы

### Классический SGD (онлайн):
$$
\large
\theta_t = \theta_{t-1} - \lambda \frac{\partial L}{\partial \theta}
$$

Количество примеров в обучающем датасете может быть достаточно большим или вообще неограничено. <br>

 - В данном случае, для одной итерации нам надо пройти по всему датасету. <br>
 - Можно обучаться "на лету" на вновь поступающих примерах.<br>
 - Нередко шаги могут уводить нас от минимума, в силу того, что мы смотрим только на один пример

### Mini-batch SGD, позволяет использовать ресурсы GPU эффективней
Делим данные на части небольшого размера $B = \{1000,256,64 ...\}$<br>
**Тогда получаем вход:<br>**
$\large x = [\overbrace{x_1, x_2, x_3, ... x_B}^{x^{\{1\}}}, \overbrace{x_{B+1}, x_{B+2}, x_{B+3}, ... x_{2*B}}^{x^{\{2\}}}, ... , x_M]  \leftarrow $ последовательность матриц размером $C \times  B$, $C$ - размер входного вектора<br>
**Выход:<br>**
$\large y = [\overbrace{y_1, y_2, y_3,...,y_B}^{y^{\{1\}}},..., ... , y_M] \leftarrow $ последовательность матриц размером $A \times  B$, $A$ - размер выходного вектора<br>

Функция ошибки усредняется не по всем примерам, а только по батчу размера $B$
$$
\overset{\wedge}{\large J}(f(x;\theta),  y) = \frac {1} {B} \sum_{i=1}^B {\large L(\tilde{y_i}, y_i)}
$$
Получаем следующую функцию отимизации для каждого шага градиентного спуска
$$
\large
\theta_t = \theta_{t-1} - \epsilon \frac{\partial \overset{\wedge}{\large J}}{\partial \theta}
$$

** Вопрос: **
Почему при обучении с полным батчем график обучения гладкий, а при использовании мини батча сильно  осциллирует?
<img src="./img/minibatch_vs_batch_gd.png" width=800>

Как выглядит градиент в пространстве весов 
1. Для режима online SGD и для обучения с GD
<img src="./img/batch_vs_sgd.png" width=800>
2. Для режима mini-batch SGD и для обучения с online SGD
<img src="./img/minibatch_vs_batch_sgd.png" width=800>

* Основной параметр алгоритма SGD – скорость обучения $ \epsilon $. На практике необходимо постепенно уменьшать ее со временем.   
- $ ε_k $ скорость обучения на k-ой итерации.

Достаточные условия сходимости SGD имеют вид:

$ \large \sum \limits_{k=1}^{\infty} \epsilon_k = \infty $

$ \large \sum \limits_{k=1}^{\infty} \epsilon_k^2 < \infty $

* На практике скорость обучения обычно уменьшают линейно до итерации с номером τ:
 $ \large ε_k = (1 – α)ε_0 + α \space ε_τ $; 
где $ α = k/τ $ . После τ-й итерации ε остается постоянным.

### SGD + Momentum (Импульсный метод)

В импульсном алгоритме вычисляется экспоненциально затухающее скользящее среднее прошлых градиентов и продолжается движение в этом направлении

* Импульсный метод призван решить две проблемы: плохую обусловленность матрицы Гессе и дисперсию стохастического градиента.

<table>
<tr>
 <td>
1. Градиент с моментом__
<img src="./img/sgd_momentum.png" width=400>
     </td><td>
2. Градиент без момента
<img src="./img/sgd.png" width=400>
</td>
</tr>
</table>

* Момент это экспоненциальное скользящее среднее по $ \large \approx \frac{1}{1-\beta}$ - последним итерациям 

$$
\Large
\begin{align}
g_t &= \nabla_{\theta} \overset{\wedge}{\large J}\left(f(x, \theta),y \right) - градиент \\
m_t &= \beta m_{t-1} - (1- \beta) g_t, \ где \  \beta - параметр\ обучения \\
m_t &= \alpha m_{t-1} - \epsilon g_t, \ альтернативная \ форма \ записи \\
\theta_t &= \theta_{t-1} + m_t
\end{align}
$$

SGD - методом нестерова

$
\large 
\begin{align}
1. \ выполняем \ проиежуточный  \ расчет \  параметров: \
 \tilde \theta = \theta + \alpha \ m_{t-1}
 \\ 
2. \ вычисляем \  градиент \ в \ промежуточной \  точке: \
g_t &= \nabla_{\theta} \overset{\wedge}{\large J}\left(f(x, \tilde \theta),y \right)
\\
3. \ вычисляем \ обновление \ скорости: \  m_t ← α  v_{t-1} – ε  g_t
\\
4. \ обновляем \ параметры: \ \theta_t &= \theta_{t-1} + m_t
 \end{align}
$

* В случае выпуклой оптимизации пакетным градиентным спуском метод Нестерова повышает скорость сходимости ошибки превышения с O(1/k) (после k шагов) до O(1/k2). К сожалению, в случае стохастического градиентного спуска метод Нестерова не улучшает скорости сходимости.

<img src="./img/4.gif" width=500>


### Алгоритмы с адаптивной скоростью обучения


### RMSProp (Root Mean Square Propagation)
#### Решаем проблемму редких признаков, ускоряя схождение.
$$
\Large
\begin{align}
v_t &- скользящее\ среднее\ по \ g^2 \\
v_t &= \beta * v_{t-1} +  (1-\beta)* g_{t}^2  \\
\theta &= \theta - \lambda \frac{1}{\sqrt{v_t + \epsilon}} g_t
\end{align}
$$
__Долго накапливает начальные значения__ 

### Adam (Adaptive Moment Estimation) 

### Adam = SGD + Momentum + RMSProp

$$
\large
\begin{align}
\begin{cases}
m_t &= \alpha \ m_{t-1} + (1 - \alpha)\ g - оценка \ момента \\ 
v_t &= \beta \ v_{t-1} +  (1-\beta)\ g_{t}^2  - оценка \ второго\ момента \\
\end{cases}
\\
\end{align}
$$
### исправляем  недостаток  RMSProp  корректируем биас на начальных  итерациях
$$
\large
\begin{align}
\hat{m_t} &= \frac{m_t}{1 - \alpha^t} \\
\hat{v_t} &= \frac{v_t}{1 - \beta^t} \\
\theta_t &= \theta_{t-1} -  \frac{\lambda}{\sqrt{\hat{v_t} + \epsilon}}\hat{m_t} \\
\ выразим\ step\_size \ через \ \alpha \ и \ \beta \\
\ step\_size =  \lambda \frac {({1 - \beta^t})^{\frac{1}{2}}}{{1 - \alpha^t}} \\ 
\theta_t &= \theta_{t-1} -  \frac{step\_size}{\sqrt{v_t + \epsilon}}\times m_t 
\end{align}
$$



<img src="./imgs/6.gif" width=500>

#### Пример градиента в седловой точке 
<img src="./img/saddle_point_algos.gif" width=500>